In [1]:
import pandas as pd
from profile_models import OUTPUT_DATABASE

/home/carol/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Transformers models characteristics

In [2]:
df_trans = pd.read_csv(OUTPUT_DATABASE)

In [3]:
df_trans

,net,layer,layer_params,depth,output_size
0,resnet50d,Conv2d,864,2,401408
1,resnet50d,BatchNorm2d,64,2,401408
2,resnet50d,ReLU,0,2,401408
3,resnet50d,Conv2d,9216,2,401408
4,resnet50d,BatchNorm2d,64,2,401408
...,...,...,...,...,...
5645,swinv2_base_window12to24_192to384_22kft1k,LayerNorm,2048,5,147456
5646,swinv2_base_window12to24_192to384_22kft1k,DropPath,0,5,147456
5647,swinv2_base_window12to24_192to384_22kft1k,Identity,0,3,147456
5648,swinv2_base_window12to24_192to384_22kft1k,LayerNorm,2048,1,147456


In [4]:
df_trans["para_by_depth"] = df_trans["layer_params"] * df_trans["depth"]

In [5]:
gp_df_trans = df_trans.groupby(["net", "layer"]).mean()

In [6]:
gp_df_trans = gp_df_trans[gp_df_trans['para_by_depth'] != 0]

In [7]:
gp_df_trans.sort_values(by=["output_size"])

layer_params  \
net                                        layer                       
resnet200d                                 Linear       2.049000e+06   
resnet50d                                  Linear       2.049000e+06   
tf_efficientnet_b7                         Linear       2.561000e+06   
vit_base_patch32_384                       Conv2d       2.360064e+06   
                                           LayerNorm    1.536000e+03   
vit_base_patch16_224                       Conv2d       5.905920e+05   
                                           LayerNorm    1.536000e+03   
swinv2_base_window12to16_192to256_22kft1k  LayerNorm    1.019170e+03   
resnet200d                                 BatchNorm2d  8.591610e+02   
                                           Conv2d       3.047216e+05   
resnet50d                                  BatchNorm2d  9.681455e+02   
                                           Conv2d       4.268003e+05   
vit_base_patch32_384                       Linear       1.750747e+06   
swinv2_large_window12to16_192to256_22kft1k LayerNorm    1.528755e+03   
vit_large_patch14_clip_224                 Conv2d       6.021120e+05   
                                           LayerNorm    2.048000e+03   
swinv2_base_window12to16_192to256_22kft1k  Linear       5.394785e+05   
vit_huge_patch14_clip_224                  Conv2d       7.526400e+05   
                                           LayerNorm    2.560000e+03   
vit_base_patch16_224                       Linear       1.750747e+06   
swinv2_base_window12to24_192to384_22kft1k  LayerNorm    1.019170e+03   
swinv2_large_window12to16_192to256_22kft1k Linear       1.205650e+06   
swinv2_base_window12to16_192to256_22kft1k  Conv2d       6.272000e+03   
swinv2_large_window12to24_192to384_22kft1k LayerNorm    1.528755e+03   
vit_large_patch14_clip_224                 Linear       3.126145e+06   
vit_large_patch14_clip_336                 Conv2d       6.021120e+05   
                                           LayerNorm    2.048000e+03   
swinv2_base_window12to24_192to384_22kft1k  Linear       5.394785e+05   
vit_huge_patch14_clip_224                  Linear       4.889886e+06   
vit_huge_patch14_clip_336                  Conv2d       7.526400e+05   
                                           LayerNorm    2.560000e+03   
swinv2_large_window12to16_192to256_22kft1k Conv2d       9.408000e+03   
swinv2_large_window12to24_192to384_22kft1k Linear       1.205650e+06   
tf_efficientnet_b7                         Conv2d       2.366704e+05   
swinv2_base_window12to24_192to384_22kft1k  Conv2d       6.272000e+03   
vit_large_patch14_clip_336                 Linear       3.126145e+06   
vit_huge_patch14_clip_336                  Linear       4.889886e+06   
swinv2_large_window12to24_192to384_22kft1k Conv2d       9.408000e+03   
tf_efficientnet_b7                         Conv2dSame   9.715200e+03   

                                                           depth  \
net                                        layer                   
resnet200d                                 Linear       1.000000   
resnet50d                                  Linear       1.000000   
tf_efficientnet_b7                         Linear       1.000000   
vit_base_patch32_384                       Conv2d       2.000000   
                                           LayerNorm    2.920000   
vit_base_patch16_224                       Conv2d       2.000000   
                                           LayerNorm    2.920000   
swinv2_base_window12to16_192to256_22kft1k  LayerNorm    4.811321   
resnet200d                                 BatchNorm2d  3.000000   
                                           Conv2d       3.004878   
resnet50d                                  BatchNorm2d  3.000000   
                                           Conv2d       3.018182   
vit_base_patch32_384                       Linear       3.938776   
swinv2_large_window12to16_192to256_22kft1k LayerNorm    4.811321   
vit_large_patch

In [10]:
df_trans["layer"].unique()

array(['Conv2d', 'BatchNorm2d', 'ReLU', 'MaxPool2d', 'Identity',
       'AvgPool2d', 'AdaptiveAvgPool2d', 'Flatten', 'Linear',
       'Conv2dSame', 'SiLU', 'Sigmoid', 'Dropout', 'LayerNorm', 'GELU',
       'Softmax', 'DropPath'], dtype=object)